# The Nature Conservancy Fisheries Monitoring Kaggle Competition

This is an attempt to compete in the above mentioned Kaggle competition using a neural network written in Keras.

In [5]:
# Lets load us some settings!
import json

print "Loading settings..."
with open('SETTINGS.json') as settings_file:
    settings = json.load(settings_file)

# Source directory for your data
source_dir = settings['source_dir']
train_dir = settings['train_dir']

print "Settings loaded!"

Loading settings...
Settings loaded!
